In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import os
import glob
from tqdm import tqdm
from joblib import Parallel, delayed
import gc

from sklearn.model_selection import train_test_split, KFold

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [2]:
class Config:
    # data_dir = '../input/optiver-realized-volatility-prediction/'
    data_dir = 'F:/SF2935/project/'
    seed = 42

In [3]:
train = pd.read_csv(Config.data_dir + 'train.csv')
train.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [4]:
train.stock_id.unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  26,  27,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
        42,  43,  44,  46,  47,  48,  50,  51,  52,  53,  55,  56,  58,
        59,  60,  61,  62,  63,  64,  66,  67,  68,  69,  70,  72,  73,
        74,  75,  76,  77,  78,  80,  81,  82,  83,  84,  85,  86,  87,
        88,  89,  90,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       118, 119, 120, 122, 123, 124, 125, 126], dtype=int64)

In [5]:
test = pd.read_csv(Config.data_dir + 'test.csv')
test.head()

,stock_id,time_id,row_id
0,0,4,0-4
1,0,32,0-32
2,0,34,0-34


In [6]:
display(train.groupby('stock_id').size())

print("\nUnique size values")
display(train.groupby('stock_id').size().unique())

stock_id
0      3830
1      3830
2      3830
3      3830
4      3830
       ... 
122    3830
123    3830
124    3830
125    3830
126    3830
Length: 112, dtype: int64


Unique size values


array([3830, 3829, 3815, 3820], dtype=int64)

In [7]:
def get_trade_and_book_by_stock_and_time_id(stock_id, time_id=None, dataType = 'train'):
    book_example = pd.read_parquet(f'{Config.data_dir}book_{dataType}.parquet/stock_id={stock_id}')
    trade_example =  pd.read_parquet(f'{Config.data_dir}trade_{dataType}.parquet/stock_id={stock_id}')
    if time_id:
        book_example = book_example[book_example['time_id']==time_id]
        trade_example = trade_example[trade_example['time_id']==time_id]
    book_example.loc[:,'stock_id'] = stock_id
    trade_example.loc[:,'stock_id'] = stock_id
    return book_example, trade_example

#### Feature engineering

In [8]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))


def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def calculate_wap1(df):
    a1 = df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']
    b1 = df['bid_size1'] + df['ask_size1']
    a2 = df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']
    b2 = df['bid_size2'] + df['ask_size2']
    
    x = (a1/b1 + a2/b2)/ 2
    
    return x


def calculate_wap2(df):
        
    a1 = df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']
    a2 = df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']
    b = df['bid_size1'] + df['ask_size1'] + df['bid_size2']+ df['ask_size2']
    
    x = (a1 + a2)/ b
    return x

def realized_volatility_per_time_id(file_path, prediction_column_name):

    stock_id = file_path.split('=')[1]

    df_book = pd.read_parquet(file_path)
    df_book['wap1'] = calculate_wap1(df_book)
    df_book['wap2'] = calculate_wap2(df_book)

    df_book['log_return1'] = df_book.groupby(['time_id'])['wap1'].apply(log_return)
    df_book['log_return2'] = df_book.groupby(['time_id'])['wap2'].apply(log_return)
    df_book = df_book[~df_book['log_return1'].isnull()]

    df_rvps =  pd.DataFrame(df_book.groupby(['time_id'])[['log_return1', 'log_return2']].agg(realized_volatility)).reset_index()
    df_rvps[prediction_column_name] = 0.6 * df_rvps['log_return1'] + 0.4 * df_rvps['log_return2']

    df_rvps['row_id'] = df_rvps['time_id'].apply(lambda x:f'{stock_id}-{x}')
    
    return df_rvps[['row_id',prediction_column_name]]

In [9]:
def get_agg_info(df):
    agg_df = df.groupby(['stock_id', 'time_id']).agg(mean_sec_in_bucket = ('seconds_in_bucket', 'mean'), 
                                                     mean_price = ('price', 'mean'),
                                                     mean_size = ('size', 'mean'),
                                                     mean_order = ('order_count', 'mean'),
                                                     max_sec_in_bucket = ('seconds_in_bucket', 'max'), 
                                                     max_price = ('price', 'max'),
                                                     max_size = ('size', 'max'),
                                                     max_order = ('order_count', 'max'),
                                                     min_sec_in_bucket = ('seconds_in_bucket', 'min'), 
                                                     min_price = ('price', 'min'),
                                                     #min_size = ('size', 'min'),
                                                     #min_order = ('order_count', 'min'),
                                                     median_sec_in_bucket = ('seconds_in_bucket', 'median'), 
                                                     median_price = ('price', 'median'),
                                                     median_size = ('size', 'median'),
                                                     median_order = ('order_count', 'median')
                                                    ).reset_index()
    
    return agg_df

In [10]:
def get_stock_stat(stock_id : int, dataType = 'train'):
    
    book_subset, trade_subset = get_trade_and_book_by_stock_and_time_id(stock_id, dataType=dataType)
    book_subset.sort_values(by=['time_id', 'seconds_in_bucket'])

    ## book data processing
    
    book_subset['bas'] = (book_subset[['ask_price1', 'ask_price2']].min(axis = 1)
                                / book_subset[['bid_price1', 'bid_price2']].max(axis = 1)
                                - 1)                               

    
    book_subset['wap1'] = calculate_wap1(book_subset)
    book_subset['wap2'] = calculate_wap2(book_subset)
    
    book_subset['log_return_bid_price1'] = np.log(book_subset['bid_price1'].pct_change() + 1)
    book_subset['log_return_ask_price1'] = np.log(book_subset['ask_price1'].pct_change() + 1)
    # book_subset['log_return_bid_price2'] = np.log(book_subset['bid_price2'].pct_change() + 1)
    # book_subset['log_return_ask_price2'] = np.log(book_subset['ask_price2'].pct_change() + 1)
    book_subset['log_return_bid_size1'] = np.log(book_subset['bid_size1'].pct_change() + 1)
    book_subset['log_return_ask_size1'] = np.log(book_subset['ask_size1'].pct_change() + 1)
    # book_subset['log_return_bid_size2'] = np.log(book_subset['bid_size2'].pct_change() + 1)
    # book_subset['log_return_ask_size2'] = np.log(book_subset['ask_size2'].pct_change() + 1)
    book_subset['log_ask_1_div_bid_1'] = np.log(book_subset['ask_price1'] / book_subset['bid_price1'])
    book_subset['log_ask_1_div_bid_1_size'] = np.log(book_subset['ask_size1'] / book_subset['bid_size1'])
    

    book_subset['log_return1'] = (book_subset.groupby(by = ['time_id'])['wap1'].
                                  apply(log_return).
                                  reset_index(drop = True).
                                  fillna(0)
                                 )
    book_subset['log_return2'] = (book_subset.groupby(by = ['time_id'])['wap2'].
                                  apply(log_return).
                                  reset_index(drop = True).
                                  fillna(0)
                                 )
    
    stock_stat = pd.merge(
        book_subset.groupby(by = ['time_id'])['log_return1'].agg(realized_volatility).reset_index(),
        book_subset.groupby(by = ['time_id'], as_index = False)['bas'].mean(),
        on = ['time_id'],
        how = 'left'
    )
    
    stock_stat = pd.merge(
        stock_stat,
        book_subset.groupby(by = ['time_id'])['log_return2'].agg(realized_volatility).reset_index(),
        on = ['time_id'],
        how = 'left'
    )
    
    stock_stat = pd.merge(
        stock_stat,
        book_subset.groupby(by = ['time_id'])['log_return_bid_price1'].agg(realized_volatility).reset_index(),
        on = ['time_id'],
        how = 'left'
    )
    stock_stat = pd.merge(
        stock_stat,
        book_subset.groupby(by = ['time_id'])['log_return_ask_price1'].agg(realized_volatility).reset_index(),
        on = ['time_id'],
        how = 'left'
    )
    stock_stat = pd.merge(
        stock_stat,
        book_subset.groupby(by = ['time_id'])['log_return_bid_size1'].agg(realized_volatility).reset_index(),
        on = ['time_id'],
        how = 'left'
    )
    stock_stat = pd.merge(
        stock_stat,
        book_subset.groupby(by = ['time_id'])['log_return_ask_size1'].agg(realized_volatility).reset_index(),
        on = ['time_id'],
        how = 'left'
    )
    
    stock_stat = pd.merge(
        stock_stat,
        book_subset.groupby(by = ['time_id'])['log_ask_1_div_bid_1'].agg(realized_volatility).reset_index(),
        on = ['time_id'],
        how = 'left'
    )
    stock_stat = pd.merge(
        stock_stat,
        book_subset.groupby(by = ['time_id'])['log_ask_1_div_bid_1_size'].agg(realized_volatility).reset_index(),
        on = ['time_id'],
        how = 'left'
    )
    
    
    stock_stat['stock_id'] = stock_id
    
    # Additional features that can be added. Referenced from https://www.kaggle.com/yus002/realized-volatility-prediction-lgbm-train/data
    
    # trade_subset_agg = get_agg_info(trade_subset)
    
    #     stock_stat = pd.merge(
    #         stock_stat,
    #         trade_subset_agg,
    #         on = ['stock_id', 'time_id'],
    #         how = 'left'
    #     )
    
    ## trade data processing 
    
    return stock_stat

def get_data_set(stock_ids : list, dataType = 'train'):

    stock_stat = Parallel(n_jobs=-1)(
        delayed(get_stock_stat)(stock_id, dataType) 
        for stock_id in stock_ids
    )
    
    stock_stat_df = pd.concat(stock_stat, ignore_index = True)

    return stock_stat_df

In [11]:
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

In [12]:
book_stock_1, trade_stock_1 = get_trade_and_book_by_stock_and_time_id(1, 5)
display(book_stock_1.shape)
display(trade_stock_1.shape)

(575, 11)

(94, 6)

In [13]:
book_stock_1.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.000754,1.001542,1.000689,1.001607,1,25,25,100,1
1,5,1,1.000754,1.001673,1.000689,1.001739,26,60,25,100,1
2,5,2,1.000754,1.001411,1.000623,1.001476,1,25,25,125,1
3,5,3,1.000754,1.001542,1.000689,1.001607,125,25,126,36,1
4,5,4,1.000754,1.001476,1.000623,1.001542,100,100,25,25,1


In [14]:
trade_stock_1.head()

,time_id,seconds_in_bucket,price,size,order_count,stock_id
0,5,28,1.002080,553,11,1
1,5,39,1.002460,8,3,1
2,5,42,1.002308,147,4,1
3,5,44,1.002788,1,1,1
4,5,51,1.002657,100,2,1


In [15]:
%%time
train_stock_stat_df = get_data_set(train.stock_id.unique(), dataType = 'train')
train_stock_stat_df.head()

CPU times: total: 2.52 s
Wall time: 3min 16s


,time_id,log_return1,bas,log_return2,log_return_bid_price1,log_return_ask_price1,log_return_bid_size1,log_return_ask_size1,log_ask_1_div_bid_1,log_ask_1_div_bid_1_size,stock_id
0,5,0.004115,0.000852,0.004106,0.002602,0.002476,25.371834,29.772642,0.015251,47.173981,0
1,11,0.001268,0.000394,0.001507,0.003793,0.003685,17.557256,16.155779,0.005999,39.853464,0
2,16,0.002719,0.000725,0.002469,0.001697,0.001792,14.737648,11.275511,0.010191,29.314081,0
3,31,0.002625,0.000861,0.002709,0.002992,0.002740,13.412586,9.774892,0.009908,26.981808,0
4,62,0.001901,0.000397,0.001932,0.002536,0.001926,23.797381,17.759397,0.005543,36.746812,0


In [16]:
train_data_set = pd.merge(train, train_stock_stat_df, on = ['stock_id', 'time_id'], how = 'left')
train_data_set.head()

,stock_id,time_id,target,log_return1,bas,log_return2,log_return_bid_price1,log_return_ask_price1,log_return_bid_size1,log_return_ask_size1,log_ask_1_div_bid_1,log_ask_1_div_bid_1_size
0,0,5,0.004136,0.004115,0.000852,0.004106,0.002602,0.002476,25.371834,29.772642,0.015251,47.173981
1,0,11,0.001445,0.001268,0.000394,0.001507,0.003793,0.003685,17.557256,16.155779,0.005999,39.853464
2,0,16,0.002168,0.002719,0.000725,0.002469,0.001697,0.001792,14.737648,11.275511,0.010191,29.314081
3,0,31,0.002195,0.002625,0.000861,0.002709,0.002992,0.002740,13.412586,9.774892,0.009908,26.981808
4,0,62,0.001747,0.001901,0.000397,0.001932,0.002536,0.001926,23.797381,17.759397,0.005543,36.746812


In [17]:
train_data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428932 entries, 0 to 428931
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   stock_id                  428932 non-null  int64  
 1   time_id                   428932 non-null  int64  
 2   target                    428932 non-null  float64
 3   log_return1               428932 non-null  float64
 4   bas                       428932 non-null  float32
 5   log_return2               428932 non-null  float64
 6   log_return_bid_price1     428932 non-null  float32
 7   log_return_ask_price1     428932 non-null  float32
 8   log_return_bid_size1      428932 non-null  float64
 9   log_return_ask_size1      428932 non-null  float64
 10  log_ask_1_div_bid_1       428932 non-null  float32
 11  log_ask_1_div_bid_1_size  428932 non-null  float64
dtypes: float32(4), float64(6), int64(2)
memory usage: 32.7 MB


In [18]:
%%time
test_stock_stat_df = get_data_set(test['stock_id'].unique(), dataType = 'test')
test_stock_stat_df

CPU times: total: 0 ns
Wall time: 65.4 ms


,time_id,log_return1,bas,log_return2,log_return_bid_price1,log_return_ask_price1,log_return_bid_size1,log_return_ask_size1,log_ask_1_div_bid_1,log_ask_1_div_bid_1_size,stock_id
0,4,0.000273,0.000557,0.000263,0.0,0.000049,1.159021,1.609438,0.000966,2.677473,0


In [19]:
test_data_set = pd.merge(test, test_stock_stat_df, on = ['stock_id', 'time_id'], how = 'left')
test_data_set.fillna(-999, inplace=True)
test_data_set

,stock_id,time_id,row_id,log_return1,bas,log_return2,log_return_bid_price1,log_return_ask_price1,log_return_bid_size1,log_return_ask_size1,log_ask_1_div_bid_1,log_ask_1_div_bid_1_size
0,0,4,0-4,0.000273,0.000557,0.000263,0.0,0.000049,1.159021,1.609438,0.000966,2.677473
1,0,32,0-32,-999.000000,-999.000000,-999.000000,-999.0,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000
2,0,34,0-34,-999.000000,-999.000000,-999.000000,-999.0,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000


In [25]:
# train_data_set.to_pickle('train_features_df.pickle')
# test_data_set.to_pickle('test_features_df.pickle')

In [20]:
x = gc.collect()

In [21]:
X_display = train_data_set.drop(['stock_id', 'time_id', 'target'], axis = 1)
X = X_display.values
y = train_data_set['target'].values

X.shape, y.shape

((428932, 9), (428932,))

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=Config.seed, shuffle=False)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((343145, 9), (85787, 9), (343145,), (85787,))

In [23]:
rs = Config.seed

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

# dt = DecisionTreeRegressor(random_state=42)
dt = DecisionTreeRegressor(
    max_depth=4,      
    # random_state=42
)
dt.fit(X_train, y_train)
preds = dt.predict(X_test)
R2 = round(r2_score(y_true=y_test, y_pred=preds), 6)
RMSPE = round(rmspe(y_true=y_test, y_pred=preds), 6)
print(f'Performance of the Decision Tree prediction: R2 score: {R2}, RMSPE: {RMSPE}')

Performance of the Decision Tree prediction: R2 score: 0.714459, RMSPE: 0.335916


In [24]:
import numpy as np
import pandas as pd
import time
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

models = {
    # 'AdaBoost': AdaBoostRegressor(
    #     estimator=DecisionTreeRegressor(max_depth=4, random_state=42),
    #     n_estimators=100,
    #     learning_rate=0.1,
    #     random_state=42
    # ),
    'Bagging': BaggingRegressor(
        estimator=DecisionTreeRegressor(max_depth=4, random_state=42),
        n_estimators=100,
        # max_samples=0.8,
        # max_features=0.8,
        random_state=42
    ),
    'RandomForest': RandomForestRegressor(
        n_estimators=100,
        max_depth=4,
        # max_features='sqrt', 
        random_state=42
    )
}


results = []

for name, model in models.items():
    print(f"Training {name}...")
    
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    y_pred = model.predict(X_test)
   
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmspe_val = rmspe(y_test, y_pred)
    
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()
    
    y_train_pred = model.predict(X_train)
    train_r2 = r2_score(y_train, y_train_pred)
    overfitting_gap = train_r2 - r2
    

    results.append({
        'Model': name,
        'R2': round(r2, 4),
        'RMSE': round(rmse, 4),
        'RMSPE': round(rmspe_val, 4),
        'Training_Time': round(training_time, 2),
        'CV_Mean_R2': round(cv_mean, 4),
        'CV_Std': round(cv_std, 4),
        'Overfitting_Gap': round(overfitting_gap, 4)
    })
    
    print(f"{name} completed in {training_time:.2f} seconds")

results_df = pd.DataFrame(results)
print("\n" + "="*80)
print("MODEL COMPARISON RESULTS")
print("="*80)
print(results_df.to_string(index=False))

print("\n" + "="*50)
print("PERFORMANCE RANKINGS")
print("="*50)

r2_rank = results_df[['Model', 'R2']].sort_values('R2', ascending=False)
print("R2 Score Ranking:")
for i, (_, row) in enumerate(r2_rank.iterrows(), 1):
    print(f"{i}. {row['Model']}: {row['R2']}")

rmspe_rank = results_df[['Model', 'RMSPE']].sort_values('RMSPE')
print("\nRMSPE Ranking (lower is better):")
for i, (_, row) in enumerate(rmspe_rank.iterrows(), 1):
    print(f"{i}. {row['Model']}: {row['RMSPE']}")


stability_rank = results_df[['Model', 'CV_Std']].sort_values('CV_Std')
print("\nStability Ranking (CV Std, lower is better):")
for i, (_, row) in enumerate(stability_rank.iterrows(), 1):
    print(f"{i}. {row['Model']}: {row['CV_Std']}")


overfit_rank = results_df[['Model', 'Overfitting_Gap']].sort_values('Overfitting_Gap')
print("\nOverfitting Resistance Ranking (gap, lower is better):")
for i, (_, row) in enumerate(overfit_rank.iterrows(), 1):
    print(f"{i}. {row['Model']}: {row['Overfitting_Gap']}")

print("\n" + "="*50)
print("COMPREHENSIVE ANALYSIS")
print("="*50)

best_r2_model = results_df.loc[results_df['R2'].idxmax()]
best_rmspe_model = results_df.loc[results_df['RMSPE'].idxmin()]
fastest_model = results_df.loc[results_df['Training_Time'].idxmin()]
most_stable_model = results_df.loc[results_df['CV_Std'].idxmin()]
least_overfitting_model = results_df.loc[results_df['Overfitting_Gap'].idxmin()]

print(f"Best R2: {best_r2_model['Model']} ({best_r2_model['R2']})")
print(f"Best RMSPE: {best_rmspe_model['Model']} ({best_rmspe_model['RMSPE']})")
print(f"Fastest Training: {fastest_model['Model']} ({fastest_model['Training_Time']}s)")
print(f"Most Stable: {most_stable_model['Model']} (CV Std: {most_stable_model['CV_Std']})")
print(f"Least Overfitting: {least_overfitting_model['Model']} (Gap: {least_overfitting_model['Overfitting_Gap']})")

print("\n" + "="*50)
print("VALIDATION OF TABLE CHARACTERISTICS")
print("="*50)

print("Noise Sensitivity (based on CV stability):")
for _, row in stability_rank.iterrows():
    sensitivity = "Low" if row['CV_Std'] < 0.05 else "Moderate" if row['CV_Std'] < 0.1 else "High"
    print(f"- {row['Model']}: {sensitivity}")

print("\nOverfitting Risk:")
for _, row in overfit_rank.iterrows():
    risk = "Low" if row['Overfitting_Gap'] < 0.05 else "Moderate" if row['Overfitting_Gap'] < 0.1 else "High"
    print(f"- {row['Model']}: {risk}")

Training Bagging...
Bagging completed in 72.44 seconds
Training RandomForest...
RandomForest completed in 169.00 seconds

MODEL COMPARISON RESULTS
       Model     R2   RMSE  RMSPE  Training_Time  CV_Mean_R2  CV_Std  Overfitting_Gap
     Bagging 0.7636 0.0014 0.3135          72.44      0.7675  0.0062           0.0110
RandomForest 0.7618 0.0014 0.3159         169.00      0.7657  0.0062           0.0117

PERFORMANCE RANKINGS
R2 Score Ranking:
1. Bagging: 0.7636
2. RandomForest: 0.7618

RMSPE Ranking (lower is better):
1. Bagging: 0.3135
2. RandomForest: 0.3159

Stability Ranking (CV Std, lower is better):
1. Bagging: 0.0062
2. RandomForest: 0.0062

Overfitting Resistance Ranking (gap, lower is better):
1. Bagging: 0.011
2. RandomForest: 0.0117

COMPREHENSIVE ANALYSIS
Best R2: Bagging (0.7636)
Best RMSPE: Bagging (0.3135)
Fastest Training: Bagging (72.44s)
Most Stable: Bagging (CV Std: 0.0062)
Least Overfitting: Bagging (Gap: 0.011)

VALIDATION OF TABLE CHARACTERISTICS
Noise Sensitivity (